# Checklist
- Criar um agent com Python
- Fazer perguntas para um CSV
- conectar um banco no bigquery
- Fazer uma pergunta que relacionada um dado do CSV com o bigquery

In [1]:
#%pip install langchain openai chromadb tiktoken tabulate sqlalchemy sqlalchemy-bigquery google-cloud-bigquery

from dotenv import load_dotenv
load_dotenv()

from langchain.agents import create_csv_agent
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_types import AgentType

In [2]:
from google.cloud import bigquery
from sqlalchemy import *
from sqlalchemy.engine import create_engine
from sqlalchemy.schema import *
import os
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.llms.openai import OpenAI
from langchain.agents import AgentExecutor

In [3]:
service_account_file = "credentials.json"
project = "quaq-plataform"
dataset = "vendas"
table = "clientes"
sqlalchemy_url = f'bigquery://{project}/{dataset}?credentials_path={service_account_file}'
print(sqlalchemy_url)
db = SQLDatabase.from_uri(sqlalchemy_url)

bigquery://quaq-plataform/vendas?credentials_path=credentials.json


In [8]:
llm = OpenAI(temperature=0, model="text-davinci-003")
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
agent_executor = create_sql_agent(
  llm=llm,
  toolkit=toolkit,
  verbose=True,
  top_k=10,
)

In [9]:
agent_executor.run("What is the name of customer with more purcheses, and why?")
# agent_executor.run("What is email of Jorge Gerhold?")



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: clientes, produtos, vendas
Thought: I should query the schema of the vendas table to see what columns I can use.
Action: sql_db_schema
Action Input: vendas
Observation: 
CREATE TABLE `vendas` (
	`id` INT64, 
	`productId` INT64, 
	`customerId` INT64, 
	`timestamp` TIMESTAMP
)

/*
3 rows from vendas table:
id	productId	customerId	timestamp
13732	1	256	2022-09-21 04:20:35.985000+00:00
4592	2	256	2023-07-22 02:36:14.011000+00:00
11980	8	256	2022-05-15 16:41:16.136000+00:00
*/
Thought: I should query the customerId and count the number of purchases for each customer.
Action: sql_db_query
Action Input: SELECT customerId, COUNT(*) as purchases FROM vendas GROUP BY customerId ORDER BY purchases DESC LIMIT 10
Observation: [(346, 23), (1813, 22), (1970, 21), (1577, 20), (1028, 20), (657, 20), (357, 20), (1889, 19), (913, 19), (912, 19)]
Thought: I now know the final answer
Final Answer: Customer 346 ha

'Customer 346 has the most purchases with 23.'